In [1]:
import pandas as pd
import numpy as np
import time

import cv2
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Optical Flow - Lucas Kanade function
#Lucas Kanade function only tracks sparse feature set. Not all points
#Gunner Farneback's algo to calculate dense optical flow

corner_tracking_params = dict(maxCorners = 10,qualityLevel=0.3, minDistance=7,blockSize=7)
lk_params = dict(winSize=(200,200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


In [41]:

cap = cv2.VideoCapture(0)

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

In [4]:
import pandas as pd
import numpy as np
import time

import cv2
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

#Optical Flow - Lucas Kanade function
#Lucas Kanade function only tracks sparse feature set. Not all points
#Gunner Farneback's algo to calculate dense optical flow

corner_tracking_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv2.VideoCapture(0)

ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **corner_tracking_params)
mask = np.zeros_like(old_frame)

while True:
    ret, new_frame = cap.read(0)
    frame_gray = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    
        
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        #mask = cv2.line(mask, (a,b),(c,d), (255,0,0), 2)
        frame = cv2.circle(new_frame,(a,b),5,(0,0,255),-1)
        
    img = cv2.add(frame,mask)         
        
    
    cv2.imshow('frame',frame)
    k = cv2.waitKey(3) & 0xff
    if k == 27:
        break
        
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
        
cv2.destroyAllWindows()
cap.release()




In [5]:
#Dense optical flow
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()
prvsImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255 

while True:
    ret, frame = cap.read()
    nextImg = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvsImg, nextImg, None,0.5,3,15,3,5,1.2,0)
    
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1], angleInDegrees=True)
    hsv_mask[:,:,0]=ang/2    
    hsv_mask[:,:,2]=cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow("frame",bgr)
    
    k = cv2.waitKey(3) & 0xff
    if k == 27:
        break
        
    prvsImg = nextImg
    
        
cv2.destroyAllWindows()
cap.release()    
